### Libraries

In [1]:
import requests 
import json
from dotenv import load_dotenv
import os
import pandas as pd

from functools import reduce
import operator
import geopandas as gdp
import shapely


from pymongo import MongoClient

In [2]:
load_dotenv()

True

### Creando los geocodes

En este caso, voy a partir de tres ubicaciones diferentes:

- España **-->** Madrid
- Paises Bajos **-->** Amsterdam
- Francia **-->** Paris 
- Canadá **-->** Montréal 

In [3]:
countries = ["madrid","paris", "ottawa"]

EN ESTA FUNCIOÓN DE AQUÍ TENDRÍA QUE MIRAR PARA METER EL TRY AND EXCEPT

In [4]:
def get_coordinates(lista):
    d = {}
    try: 
        for country in countries:
            locations = requests.get(f'https://geocode.xyz/{country}?json=1')
            locations = locations.json()
            d[country] = locations
    except:
        return "Error getting data"
        
    return d     

In [5]:
c = get_coordinates(countries)

In [6]:
c['madrid']
for k, v in c.items():
    print(v.keys())

dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])


In [7]:
def exctract_coordinates(dictionary):
    dict_empty = {}
    for k, v in c.items():
        coor = [float(v["longt"]),float(v["latt"])]
        dict_empty[k] = coor
    return dict_empty

In [8]:
coordinates = exctract_coordinates(c)

In [9]:
coordinates

{'madrid': [-3.6793, 40.42955],
 'paris': [2.3428, 48.85756],
 'ottawa': [-75.70531, 45.35917]}

### Llamadas a la API

meter un parametro de radio = metros desde la ubicación dada
meter el parametro de limit para que no me pete, tiene un limite a la hora

In [10]:
token1= os.getenv("token1")
token2 = os.getenv("token2")

In [11]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

#### Llamadas a la API

#####  Codes to queries

Dentro de todas las queries hay algunas que tienen que estar muy cerca o otras que da igual que estén un pco más lejos, lo que haré separarlas por grupos

In [207]:
queries = ["Preschool", "Vegan Restaurant", "Bar", "Airport Terminal"]

##### Setting the parameters

In [208]:
def datos_madrid (*args):
    '''
    descripcion de lo que hace la función aquí
    '''  
    d = {}
    
    for i in args: 
        parametros = {"client_id" : token1,
                  "client_secret" : token2,
                  "v": "20180323",
                  "ll": f"{coordinates['madrid'][1]},{coordinates['madrid'][0]}", ## esto lo puedo hacer con kwargs
                  "query":i,
                  "limit": 100}  

        resp = requests.get(url= url_query, params=parametros)
        data = json.loads(resp.text)
        d[i] = data
    return d

In [209]:
results = datos_madrid(*["Preschool", "Vegan Restaurant", "Bar", "Airport"])

In [210]:
results.keys()#salen como resultado 7 diccionarios de cada una de las queries que he pedido. Despues de esto habría que 
#separar cada uno de los diccionarios y proceder a su limpieza

dict_keys(['Preschool', 'Vegan Restaurant', 'Bar', 'Airport'])

**Para separar los distintos diccionarios que he genarado con la función anterior:**

In [211]:
def create_separate_dictionaries(result):
    for key, values in result.items():
        if key == "Preschool":
            school = result.get(key)

        elif key == "Vegan Restaurant":
            vegan = result.get(key)
            
        elif key == "Bar":
            bar = result.get(key)
        
        else:
            airport = result.get(key) 
            
    return school, vegan, bar, airport

In [212]:
school, vegan, bar, airport = create_separate_dictionaries(results)

In [213]:
args = [school, vegan, bar, airport]

### Limpieza de los resultados


In [214]:
def cleaning_dictionaries(*args):
    school = args[0]
    for key, values in school.items():

        response = school.get('response')
        decoded = response.get('groups')[0]
        final_school = decoded.get('items')

    vegan = args[1]
    for key, values in vegan.items():

        response = vegan.get('response')
        decoded = response.get('groups')[0]
        final_vegan = decoded.get('items')
        
    bar = args[2]
    for key, values in bar.items():

        response = bar.get('response')
        decoded = response.get('groups')[0]
        final_bar = decoded.get('items')

    airport = args[3]
    for key, values in airport.items():

        response = airport.get('response')
        decoded = response.get('groups')[0]
        final_airport = decoded.get('items')

    return final_school, final_vegan, final_bar, final_airport

In [215]:
school_results, vegan_results, bar_results, airport_results = cleaning_dictionaries(*args)               

### Creating dataframes

In [216]:
mapa_nombre = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [217]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

####  For school

In [218]:
data_schools = []
for dic in school_results:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_schools.append(paralista)

In [219]:
data_schools[0]

{'name': "ABC Centre d'Estudis", 'latitud': 40.4027257, 'longitud': -3.6748938}

####  For vegan

In [220]:
data_vegan = []
for dic in vegan_results:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_vegan.append(paralista)
data_vegan[0]

{'name': 'Planet Organic Food&Life',
 'latitud': 40.428808,
 'longitud': -3.681333}

#### For bar

In [221]:
data_bar = []
for dic in bar_results:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_bar.append(paralista)

In [222]:
data_vegan[0]

{'name': 'Planet Organic Food&Life',
 'latitud': 40.428808,
 'longitud': -3.681333}

#### For airport

In [223]:
data_airport = []
for dic in airport_results:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_airport.append(paralista)

In [224]:
data_airport

[{'name': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
  'latitud': 40.481654479394656,
  'longitud': -3.578023910522461},
 {'name': 'AIRPORT TAVERN',
  'latitud': 40.45401795837524,
  'longitud': -3.678820633312091},
 {'name': 'A|X Madrid Airport Barajas',
  'latitud': 40.41414301418716,
  'longitud': -3.7030277251869843},
 {'name': 'Parada Atocha Renfe - Línea Exprés Aeropuerto',
  'latitud': 40.40672410393115,
  'longitud': -3.689762820472258},
 {'name': 'Gate A8 – T1 (MAD)',
  'latitud': 40.46138669618143,
  'longitud': -3.5747534179940046},
 {'name': 'Gate A2 – T1 (MAD)',
  'latitud': 40.46092574171438,
  'longitud': -3.575276791831805},
 {'name': 'Aeropuerto de Madrid-Cuatro Vientos (MCV)',
  'latitud': 40.369272082642745,
  'longitud': -3.778696060180664},
 {'name': 'Gate H8 – T4 (MAD)',
  'latitud': 40.48713225722167,
  'longitud': -3.5901702504804733},
 {'name': 'Llegadas Sala 10',
  'latitud': 40.4912994992147,
  'longitud': -3.59

###  Saving dataframes

#### School query

In [186]:
df_pets = pd.DataFrame(data_schools)
df_pets.to_csv("school.csv", index = False)
df_pets.head()

,name,latitud,longitud
0,ABC Centre d'Estudis,40.402726,-3.674894
1,Guardería Chiqui Tin Moratalaz,40.409716,-3.653979
2,Foodies in the City,40.421212,-3.717058


#### Vegan query

In [189]:
df_gym = pd.DataFrame(data_vegan)
df_gym.to_csv("vegan.csv", index = False)
df_gym.head()

,name,latitud,longitud
0,Planet Organic Food&Life,40.428808,-3.681333
1,Swagat Indian Restaurant,40.424513,-3.683114
2,Roots,40.424572,-3.695587
3,Bunny's Deli,40.423935,-3.697158
4,La Galette 2 - Casa Escondida,40.423449,-3.692540


In [ ]:
def create_separate_dictionaries (dictonary_queries):
    filterByKey = lambda keys: {x: results[x] for x in keys}
    for key, values in dictonary_queries.items():
        
        return filterByKey([values])
    
    results *args